### Initial imports

In [48]:
import json
from pathlib import Path
import pandas as pd

### Exploring Dataset

In [ ]:

path = Path("..") / "src" / "data" / "legal_text_dataset.json"
print(f"Loading from: {path}")

with open(path, encoding="utf-8") as f:
    data = json.load(f)

def extract_from_results(results_list):
    out = []
    for res in results_list:
        # 'res' may be a mapping with 'value' key
        val = res.get("value") if isinstance(res, dict) else None
        if not isinstance(val, dict):
            continue
        ch = val.get("choices")
        if ch:
            out.extend(ch)
    return out

rows = []
for item in data:
    text = item.get("data", {}).get("text")

    choices = []

    for ann in item.get("annotations", []):
        results = ann.get("result", [])
        choices.extend(extract_from_results(results))

    if not choices:
        for pred in item.get("predictions", []):
            results = pred.get("result", [])
            choices.extend(extract_from_results(results))

    if choices:
        seen = set()
        unique = []
        for c in choices:
            if c not in seen:
                seen.add(c)
                unique.append(c)
        choices = unique
    else:
        choices = None

    rows.append({"text": text, "choices": choices[0]})

df = pd.DataFrame(rows)



Loading from: ..\src\data\legal_text_dataset.json


In [50]:
df.head()

,text,choices
0,A Mobilalkalmazás szellemi tulajdonjoga (a tov...,3-Többé/kevésbé megértem
1,A felhasználók kötelesek a Mobilalkalmazás alk...,4-Érthető
2,A Felhasználó bármikor kérheti regisztrációján...,5-Könnyen érthető
3,"Tilos a Mobilalkalmazás visszafejtéssel, vagy ...",4-Érthető
4,E jog bárminemű megsértése esetén eljárást kez...,2-Nehezen érthető


In [51]:
map = {}

for uv in df['choices'].unique():
    if uv not in map:
        map[uv] = int(uv[0])-1

map

{'3-Többé/kevésbé megértem': 2,
 '4-Érthető': 3,
 '5-Könnyen érthető': 4,
 '2-Nehezen érthető': 1,
 '1-Nagyon nehezen érthető': 0}

In [52]:
df['label']=df['choices'].apply(lambda x: map.get(x))
df.head()

,text,choices,label
0,A Mobilalkalmazás szellemi tulajdonjoga (a tov...,3-Többé/kevésbé megértem,2
1,A felhasználók kötelesek a Mobilalkalmazás alk...,4-Érthető,3
2,A Felhasználó bármikor kérheti regisztrációján...,5-Könnyen érthető,4
3,"Tilos a Mobilalkalmazás visszafejtéssel, vagy ...",4-Érthető,3
4,E jog bárminemű megsértése esetén eljárást kez...,2-Nehezen érthető,1


In [53]:
df['choices'].isna().sum()

0

In [54]:
df['label'].value_counts()

label
3    46
4    44
2    41
1     8
0     5
Name: count, dtype: int64